In [10]:
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup  
import json
import csv
import numpy as np
import matplotlib.pyplot as plt      
from datetime import datetime
from datetime import timedelta
import math
import re


In [11]:
df = pd.read_csv("Data/Real_Estate_TLV_Step_2.csv",index_col=0)

In [12]:
def distance_from_the_see_TLV(X_coordinate, Y_coordinate):
    north_x = 180471
    north_y = 672391
    south_x = 177333
    south_y = 663016
    
    m = (south_y - north_y) / (south_x - north_x)   
    b = north_y - (m * north_x)
    
    numerator = abs(m * X_coordinate - Y_coordinate + b)
    denominator = math.sqrt(m**2 + 1)
    return numerator / denominator

def update_centroid(df, col_name):
    new_col = []
    long = []
    lat = []
    pattern = re.compile(r"y': (.*?), 'x': (.*?), 'geometryT")
    
    for i in range(df.shape[0]):
        matches = re.search(pattern, df[col_name][i])
        x_val = matches.group(2)
        y_val = matches.group(1)
        corr = {'X': x_val, 'Y': y_val}
        new_col.append(corr)
        lat.append(x_val)
        long.append(y_val)

        
    df["centroid"] = new_col
    df['long'] = long
    df['lat'] = lat
    return df


In [13]:
def Distance_from_sea_TLV(df):
    new_column = []
    df.dropna(subset=['centroid'] ,inplace=True)
    df.reset_index(drop=True,inplace =True)
    for index in range(df.shape[0]):
        x_val = float(df['centroid'][index]["X"])
        y_val = float(df['centroid'][index]["Y"])
        distance = distance_from_the_see_TLV(x_val,y_val)
        new_column.append(distance)

    df["Distance_From_Sea"] = new_column
    df['Distance_From_Sea'] = df['Distance_From_Sea'].astype(int)

    return df


In [25]:
def add_price_per_meter(df):
    price_per_meter= ((df['DEALS_DEALAMOUNT'] / df['ASSETMETER']).values)
    price_per_meter = [int(i) for i in price_per_meter]
    df.insert(6,'PRICE_PER_METER',price_per_meter, allow_duplicates=True)
    return df


In [2]:
df = update_centroid(df,"centroid")
df = Distance_from_sea_TLV(df) 
df = add_price_per_meter(df)
df = add_avg_salary(df)  
df.to_csv('Data/Real_Estate_TLV_Step_3.csv') 

